The purpose of the script is to automate the process of selecting land for investment. The area available for investment was calculated after subtracting the area of ​​third class soils and organic soils. The distance of the investment area from the main power supply point was also calculated.

In [ ]:
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import glob
import os
import shutil
from osgeo import osr

In [5]:
!pip install gdal

In [ ]:
from osgeo import gdal

In [13]:
os.chdir("/content/drive/MyDrive/Dane z Kaggla/GDAL/SHP/Project")
!ls

Land_Parcels.cpg      Main_power_point.prj  Organic_soil.shx		  Soil3_class.dbf
Land_Parcels.dbf      Main_power_point.shp  Protected_Landscape_Area.cpg  Soil3_class.prj
Land_Parcels.prj      Main_power_point.shx  Protected_Landscape_Area.dbf  Soil3_class.shp
Land_Parcels.shp      Organic_soil.cpg	    Protected_Landscape_Area.prj  Soil3_class.shx
Land_Parcels.shx      Organic_soil.dbf	    Protected_Landscape_Area.shp
Main_power_point.cpg  Organic_soil.prj	    Protected_Landscape_Area.shx
Main_power_point.dbf  Organic_soil.shp	    Soil3_class.cpg


In [19]:
import geopandas as gpd
from shapely.ops import nearest_points

def process_land_parcels(land_parcels_shp, landscape_protected_area_shp, organic_soil_shp, soil3_class_shp, main_power_point_shp, buffer_distance=100):
    # Load the shapefiles
    land_parcels = gpd.read_file(land_parcels_shp)
    landscape_protected_area = gpd.read_file(landscape_protected_area_shp)
    organic_soil = gpd.read_file(organic_soil_shp)
    soil3_class = gpd.read_file(soil3_class_shp)
    main_power_point = gpd.read_file(main_power_point_shp)

    # Calculate buffer around Landscape Protected Area
    landscape_buffer = landscape_protected_area.buffer(buffer_distance)

    # Cut area of Land Parcels using the buffer
    land_parcels_cut = gpd.overlay(land_parcels, gpd.GeoDataFrame(geometry=landscape_buffer, crs=landscape_protected_area.crs), how='difference')

    # Cut area of Land Parcels which intersect with Organic Soil and Soil III Class
    land_parcels_cut = gpd.overlay(land_parcels_cut, organic_soil, how='difference')
    land_parcels_cut = gpd.overlay(land_parcels_cut, soil3_class, how='difference')

    # Calculate the area of the rest of Land Parcels
    land_parcels_cut['area'] = land_parcels_cut.geometry.area

    # Calculate distance from Land Parcels to the nearest Main Power Point
    def calculate_distance_to_nearest_point(row, points):
        nearest_point = nearest_points(row.geometry, points.unary_union)
        return nearest_point[0].distance(nearest_point[1])  # Distance between the nearest points

    main_power_points = main_power_point.geometry
    land_parcels_cut['distance_to_main_power_point'] = land_parcels_cut.apply(calculate_distance_to_nearest_point, points=main_power_points, axis=1)

    # Save the resulting GeoDataFrame to a new shapefile
    output_shapefile = 'Land_Parcels_processed.shp'
    land_parcels_cut.to_file(output_shapefile, driver='ESRI Shapefile')
    print(f"Processed shapefile saved to {output_shapefile}")

# Example usage
land_parcels_shp = 'Land_Parcels.shp'
landscape_protected_area_shp = 'Protected_Landscape_Area.shp'
organic_soil_shp = 'Organic_soil.shp'
soil3_class_shp = 'Soil3_class.shp'
main_power_point_shp = 'Main_power_point.shp'

process_land_parcels(land_parcels_shp, landscape_protected_area_shp, organic_soil_shp, soil3_class_shp, main_power_point_shp)


<ipython-input-19-8938ff265fa7>:35: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  land_parcels_cut.to_file(output_shapefile, driver='ESRI Shapefile')


Processed shapefile saved to Land_Parcels_processed.shp
